In [39]:
#convolutional network on MNIST ~99.2% accuracy

'''
-two convolutions w/ relu
-two fc
-dropout

'''

'\n-two convolutions w/ relu\n-two fc\n-dropout\n\n'

In [40]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

In [41]:
#importing data
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [42]:
x = tf.placeholder(tf.float32, shape=[None, 784])
y_ = tf.placeholder(tf.float32, shape=[None, 10])

In [43]:
#defining helper functions to create weight and bias variables
def weight_create(shape):
    init = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(init)

def bias_create(shape):
    init = tf.constant(value=0.1, shape=shape)
    return tf.Variable(init)

In [44]:
#creating helper functions for pooling and convolution
def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1,1,1,1], padding='SAME')

def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1,2,2,1], strides = [1,2,2,1], padding = 'SAME')

In [45]:
#defining layers (conv-> pool (x2))

#conv + ReLU 1 
w_conv1 = weight_create([5, 5, 1, 32])
b_conv1 = bias_create([32])

#conv + ReLU 2
w_conv2 = weight_create([5, 5, 32, 64])
b_conv2 = bias_create([64])

#fc 3
w_fc3 = weight_create([7 * 7 * 64, 1024])
b_fc3 = bias_create([1024])

#fc 5 (FINAL)
w_fc4 = weight_create([1024, 10])
b_fc4 = bias_create([10])

In [46]:
#computing output from LAYER 1

#reshaping input image to 4d tensor with img height and img width as dim 2 and 3
x_image = tf.reshape(x, [-1, 28, 28, 1])

#passing tensor through
h_conv1 = tf.nn.relu(conv2d(x_image, w_conv1) + b_conv1)
h_pool1 = max_pool_2x2(h_conv1)

In [47]:
#computing output from LAYER 2
h_conv2 = tf.nn.relu(conv2d(h_pool1, w_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)

In [48]:
#computing output from LAYER 3

#reshaping to fit
h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])
h_fc3 = tf.nn.relu(tf.matmul(h_pool2_flat, w_fc3) + b_fc3)

In [49]:
#dropout 4
keep_prob = tf.placeholder(tf.float32)
h_fc3_drop = tf.nn.dropout(h_fc3, keep_prob=keep_prob)

In [50]:
#computing class probabilities from fc LAYER 4
y_conv = tf.matmul(h_fc3_drop, w_fc4) + b_fc4

In [51]:
#variables to run the model and evaluate

#log-loss
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(y_conv, y_))

#optimizer ADAM > SGD (adds momentum, separate lr for each parameter)
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)

#checking if highest activation is equal --> correct prediction
correct_pred = tf.equal(tf.argmax(y_conv, 1), tf.arg_max(y_, 1))

#compute accuracy metric
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

In [ ]:
#training vars
batch_size = 50
dropout_prob = 0.5

#preparing initialization of all variables
init = tf.global_variables_initializer()

#begin training
with tf.Session() as sess:
    sess.run(init)
    
    for i in range(20000):
        batch = mnist.train.next_batch(batch_size)
        #printing
        if i%100 == 0:
            train_accuracy = accuracy.eval(feed_dict={x:batch[0], y_:batch[1], keep_prob: 1.0})
            print("step= ", i, "; training accuracy= ", train_accuracy)
        train_step.run(feed_dict={x: batch[0], y_: batch[1], keep_prob: dropout_prob})

#done training; evaluating on test set
print(accuracy.eval(feed_dict={x: mnist.test.images, y_: mnist.test.labels}))